In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI

In [1]:
!pip install langchain  

In [2]:
!pip install langchain-openai


     |████████████████████████████████| 226 kB 16.8 MB/s eta 0:00:01
     |████████████████████████████████| 55 kB 4.1 MB/s  eta 0:00:01
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.0.90
    Uninstalling langsmith-0.0.90:
      Successfully uninstalled langsmith-0.0.90


In [4]:
# Few Shot Examples
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
        "input": "Jan Sindel’s was born in what country?",
        "output": "what is Jan Sindel’s personal history?",
    },
]
# We now transform these to example messages
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

In [5]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        # Few shot examples
        few_shot_prompt,
        # New question
        ("user", "{question}"),
    ]
)

In [11]:
question_gen = prompt | ChatOpenAI(temperature=0) | StrOutputParser()

In [7]:

import openai

openai.organization = 
openai.api_key = 

In [10]:
import os

os.environ['OPENAI_API_KEY'] = 

In [24]:
question = "convert 3560 billion usd to lakh crore inr"

In [25]:
question_gen.invoke({"question": question})

Task was destroyed but it is pending!
task: <Task pending name='Task-24' coro=<AsyncCurl._force_timeout() done, defined at /home/ubuntu/.local/lib/python3.8/site-packages/curl_cffi/aio.py:164> wait_for=<Future cancelled>>


'what is the conversion rate between USD and INR?'

In [26]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

search = DuckDuckGoSearchAPIWrapper(max_results=4)


def retriever(query):
    return search.run(query)

In [15]:
!pip install -U duckduckgo-search

     |████████████████████████████████| 6.9 MB 16.6 MB/s eta 0:00:01
     |████████████████████████████████| 8.0 MB 49.5 MB/s eta 0:00:01
  Attempting uninstall: nest-asyncio
    Found existing installation: nest-asyncio 1.5.9
    Uninstalling nest-asyncio-1.5.9:
      Successfully uninstalled nest-asyncio-1.5.9


In [27]:
retriever(question)

'based on the Feb. 11, 2024 0:0:3 exchange rate from openexchangerates.org. USD to INR Conversion Table How to Convert USD to INR 1 United States Dollar = 83.01474 Indian Rupee 1 Indian Rupee = 0.0120460535 United States Dollar Example: convert 15 United States Dollar to Indian Rupee: Quick Conversions from United States Dollar to Indian Rupee : 1 USD = 83.014146 INR Last Updated 2/9/2024 12:50:52 PM Currency converter to convert from United States Dollar (USD) to Indian Rupee (INR) including the latest exchange rates, a chart showing the exchange rate history for the last 120-days and information about the currencies. Money Transfer Currency Converter USD To INR 1 USD To INR Convert United States Dollar To Indian Rupee 1 USD = 83.01474 INR Feb 11, 2024 10:41 UTC Send Money Check the currency... The USD conversion factor has 6 significant digits. Large amounts of Rupees are expressed in lakh rupees or crore rupees. A Lakh Rupee is one hundred thousand rupees and a crore rupee is ten mi

In [28]:
retriever(question_gen.invoke({"question": question}))

"Current exchange rate US DOLLAR (USD) to INDIAN RUPEE (INR) including currency converter, buying & selling rate and historical conversion chart. United States Dollar to Indian Rupee 1 USD = 83.01474 INR Feb 11, 2024 10:41 UTC If you've got a trip to India coming up, you may want to convert some of your money into rupees before you... 1.00 USD = 83.037564 INR. Feb 11, 2024 11:02 UTC. View USD Rates Table. View INR Rates Table. View USD / INR Graphs. 1. Configure Converter. ↔. Currency Calculator. Currency Converter INR To USD 1 INR To USD Convert Indian Rupee To United States Dollar 1 INR = 0.01205 USD Feb 08, 2024 23:56 UTC Send Money Check the currency rates against..."

In [29]:

from langchain import hub

response_prompt = hub.pull("langchain-ai/stepback-answer")

In [20]:
!pip install langchainhub

In [30]:
chain = (
    {
        # Retrieve context using the normal question
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        # Retrieve context using the step-back question
        "step_back_context": question_gen | retriever,
        # Pass on the question
        "question": lambda x: x["question"],
    }
    | response_prompt
    | ChatOpenAI(temperature=0)
    | StrOutputParser()
)

In [31]:
chain.invoke({"question": question})

Task was destroyed but it is pending!
task: <Task pending name='Task-20' coro=<AsyncCurl._force_timeout() running at /home/ubuntu/.local/lib/python3.8/site-packages/curl_cffi/aio.py:168> wait_for=<Future cancelled>>


'To convert 3560 billion USD to lakh crore INR, we need to first convert USD to INR and then convert the resulting amount to lakh crore.\n\nAccording to the given exchange rate of 1 USD = 83.01474 INR, we can calculate the equivalent amount in INR:\n\n3560 billion USD * 83.01474 INR/USD = 295,091.8544 billion INR\n\nNow, to convert billion INR to lakh crore INR, we need to divide the amount by 100,000 (since 1 lakh crore is equal to 100,000 crore):\n\n295,091.8544 billion INR / 100,000 = 2,950.918544 lakh crore INR\n\nTherefore, 3560 billion USD is equivalent to 2,950.918544 lakh crore INR.'